In [1]:
import numpy as np
import math
from triple_integral import triple_integral
import healpy as hp
import matplotlib.pyplot as plt

In [2]:
#read data
m = hp.read_map('../COM_CMB_IQU-smica_2048_R3.00_full.fits', ('I_STOKES_INP', ))
nside = 2048
resolution = hp.nside2resol(nside, arcmin=True)/60.0
print(f'NSIDE = {nside}')
print(f'Resolution = {resolution:.2}')
print(f'Number pixel = {hp.nside2npix(nside)}')

NSIDE = 2048
Resolution = 0.029
Number pixel = 50331648


In [3]:
#Calc correlation
p = np.pi
c = 1e12/(8.0*p*p) # Normalaze and converted in uK^2 units

In [4]:
def prod(b, f, t, a):
    
    t2 = 0.0
    f2 = 0.0
    
    if math.fabs(t)<1e-10:
        t2 = a
        den = math.tan(b-f)
        if math.fabs(den)<1e-10: f2 = math.pi/2.0
        else : f2 = math.atan(1.0/den)
    else:
        st = math.sin(t)
        sf = math.sin(f)
        sa = math.sin(a)
        sb = math.sin(b)
        ct = math.cos(t)
        cf = math.cos(f)
        ca = math.cos(a)
        cb = math.cos(b)
        ta = math.tan(a)
        
        t2 = math.cos(-st*sa*sb + ct*ca)
        den = st*cf + ta*(ct*cf*sb - sf*cb)
        f2 = math.tan((st*sf + ta*(ct*sf*sb + cf*cb))/float(den))
    
    dT1 = m[hp.ang2pix(nside, t, f)]
    dT2 = m[hp.ang2pix(nside, t2, f2)]
    
    return dT1*dT2*math.sin(t)

In [5]:
n = 200
vecA = np.linspace(1e-3, p-1e-3, n)
correlation = np.zeros(n)

for i in range(n):
    correlation[i] = c*triple_integral(prod, [0, 2*p, 0, 2*p, 0, p], 7, args=(vecA[i]))
    print(f'{vecA[i]:.6e} {correlation[i]}')

1.000000e-03 43.282235891581756
1.677685e-02 42.93809903409909


In [ ]:
plt.plot(vecA, correlation, 'o-')
plt.margins(0, 1)
plt.xlabel('Angulo')
plt.ylabel('Correlation')
plt.grid()
plt.subplots_adjust(lef=0.1, bottom=0.1, right=0.96, top=0.97)
plt.show()